In [8]:
%%capture
!pip install tiktoken==0.3.3
!pip install tqdm

In [10]:
!pip install transformers


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from tqdm import tqdm
import tiktoken
import requests
import logging
import os

logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [4]:
tokenizer = tiktoken.get_encoding('cl100k_base')
DOC_DIR_PATH = './JY_docs'
CHUNK_SIZE = 256

In [5]:
def doc_iterator(dir_path: str):
    for root, _, filenames in os.walk(dir_path):
        for filename in filenames:
            file_path = os.path.join(root, filename)
            if os.path.isfile(file_path):
                with open(file_path, 'r') as file:
                    file_contents = file.read()
                    yield filename, file_contents

In [71]:
import json
import boto3
import numpy as np

smr_client = boto3.client("sagemaker-runtime")

def get_st_embedding(smr_client, text_input):
    endpoint_name = "st-paraphrase-mpnet-base-v2-2023-04-14-04-17-29-625-endpoint"
    parameters = {
      #"early_stopping": True,
      #"length_penalty": 2.0,
      "max_new_tokens": 50,
      "temperature": 0,
      "min_length": 10,
      "no_repeat_ngram_size": 2
    }

    response_model = smr_client.invoke_endpoint(
                EndpointName=endpoint_name,
                Body=json.dumps(
                {
                    "inputs": [text_input],
                    "parameters": parameters
                }
                ),
                ContentType="application/json",
            )
    
    json_str = response_model['Body'].read().decode('utf8')
    json_obj = json.loads(json_str)
    embeddings = json_obj["sentence_embeddings"]
    
    return embeddings[0]

def calulate_cosine(vector1,vector2):
    """
    Calculate cosine similarity between two vectors
    """
    return np.dot(vector1,vector2)/(np.linalg.norm(vector1)*np.linalg.norm(vector2))

def calulate_semantic_distance(smr_client, q_str, a_str, get_emb_func):
    q_vec = get_emb_func(smr_client, q_str)
    a_vec = get_emb_func(smr_client, a_str)
    return calulate_cosine(q_vec, a_vec)

In [29]:
def segment_doc_by_paragraph(input_path, tokenizer):
    paragraphs = []
    paragraph_embeddings = []
    q_line_vec_arr = []

    for doc_name, doc in tqdm(doc_iterator(DOC_DIR_PATH)):
        if doc_name == "JY_FAQ.txt":
            lines = doc.splitlines()
            max_len = len(lines)
            print(f"max_len : {max_len}")

            q_line_vec_arr = [ (line, get_st_embedding(smr_client, line)) for line in lines if line.startswith('Question') ]

            for line_idx in range(1, len(lines), 3):
                print(f"line_idx: {line_idx}")

                paragraph = '\n'.join(lines[line_idx:line_idx+3])
                # print(paragraph)
                paragraphs.append(paragraph)
                paragraph_emb = get_st_embedding(smr_client, paragraph)
                # # print(paragraph_emb)
                paragraph_embeddings.append(paragraph_emb)
                
    return paragraphs, paragraph_embeddings, q_line_vec_arr

In [30]:
paragraphs, paragraph_embeddings, q_line_vec_arr = segment_doc_by_paragraph('./JY_docs', tokenizer)

0it [00:00, ?it/s]

max_len : 1152
line_idx: 1
line_idx: 4
line_idx: 7
line_idx: 10
line_idx: 13
line_idx: 16
line_idx: 19
line_idx: 22
line_idx: 25
line_idx: 28
line_idx: 31
line_idx: 34
line_idx: 37
line_idx: 40
line_idx: 43
line_idx: 46
line_idx: 49
line_idx: 52
line_idx: 55
line_idx: 58
line_idx: 61
line_idx: 64
line_idx: 67
line_idx: 70
line_idx: 73
line_idx: 76
line_idx: 79
line_idx: 82
line_idx: 85
line_idx: 88
line_idx: 91
line_idx: 94
line_idx: 97
line_idx: 100
line_idx: 103
line_idx: 106
line_idx: 109
line_idx: 112
line_idx: 115
line_idx: 118
line_idx: 121
line_idx: 124
line_idx: 127
line_idx: 130
line_idx: 133
line_idx: 136
line_idx: 139
line_idx: 142
line_idx: 145
line_idx: 148
line_idx: 151
line_idx: 154
line_idx: 157
line_idx: 160
line_idx: 163
line_idx: 166
line_idx: 169
line_idx: 172
line_idx: 175
line_idx: 178
line_idx: 181
line_idx: 184
line_idx: 187
line_idx: 190
line_idx: 193
line_idx: 196
line_idx: 199
line_idx: 202
line_idx: 205
line_idx: 208
line_idx: 211
line_idx: 214
line_idx: 217

2it [04:45, 142.51s/it]


In [32]:
len(q_line_vec_arr)

384

In [34]:
print("start to calulate similiarity")
def retrieval_context(q_line_vec_arr, paragraph_embeddings):
    for q_doc, q_vec in q_line_vec_arr:
        max_cos = 0.0
        a_doc = ""
        for idx in range(len(paragraphs)):
            cos_val = calulate_cosine(q_vec, paragraph_embeddings[idx])
            if cos_val > max_cos:
                max_cos = cos_val
                a_doc = paragraphs[idx]
        yield q_doc, a_doc, max_cos

start to calulate similiarity


In [83]:
def LLM_generate(smr_client, prompt):
    parameters = {
      # "early_stopping": True,
      "length_penalty": 100.0,
      "max_new_tokens": 200,
      "temperature": 0,
      "min_length": 20,
      "no_repeat_ngram_size": 200
    }
    # endpoint_name='bloomz-7b1-mt-2023-04-13-11-02-25-553-endpoint'
    endpoint_name='bloomz-7b1-2023-04-17-14-48-45-941-endpoint'
    
    response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompt,
                "parameters": parameters
            }
            ),
            ContentType="application/json",
        )

    print(response_model['Body'].read().decode('utf8'))

In [107]:
prompt="""游戏《口袋奇兵》的FAQ文档中提到
Question: 我怎么迁移区服，需要什么道具资源
Answer: 开启迁出功能的服务器可以在世界地图（能看到各个服务器的那个地图）点击左下角第二个按钮查看迁服列表，点击 准备调动 之后会显示相关说明和所需调动申请书的数量哦~
， 请严格按照FAQ参考文档提取"Question: 我怎么迁移区服，需要什么道具资源"的答案
Answer: """

prompt2= """
我是《口袋奇兵》游戏的客服，客户对我们说"你们游戏好傻逼啊！我要去炸你们公司", 请帮助我礼貌的回答他。答: 
"""

LLM_generate(smr_client, prompt2)


{
  "outputs":"\n我是《口袋奇兵》游戏的客服，客户对我们说\"你们游戏好傻逼啊！我要去炸你们公司\", 请帮助我礼貌的回答他。答: \n客服: 抱歉，您对我们的游戏有任何意见或建议，欢迎您随时联系我们，我们会在第一时间回复您。"
}


In [106]:
prompt_template = "游戏《口袋奇兵》的FAQ文档中提到\n {}， 请严格按照FAQ参考文档提取\"{}\"的答案\nAnswer: "

for q_doc, a_doc, max_cos in retrieval_context(q_line_vec_arr, paragraph_embeddings):
    prompt = prompt_template.format(a_doc, q_doc)
    LLM_generate(smr_client, prompt)

{
  "outputs":"游戏《口袋奇兵》的FAQ文档中提到\n Question: 如何开启装饰物商店功能？\nAnswer: 您好，所在服开服时间≥84天或已经开过一次曙光之地活动后才可以在金融中心里看到装饰物商店页签，且需要满足玩家基地等级≥60级后才可以解锁装饰物商店功能。\n， 请严格按照FAQ参考文档提取\"Question: 如何开启装饰物商店功能？\"的答案\nAnswer: 玩家基地等级≥60级后才可以解锁装饰物商店功能。"
}
{
  "outputs":"游戏《口袋奇兵》的FAQ文档中提到\n Question: 满足什么条件才能进行全息幻视?\nAnswer: 您好，全息幻视需要满足以下两个基本条件：   1)您已经拥有了想要幻视的目标城市外观，且当前城市外观和目标城市外观都需要是永久城市外观； 2)目标城市外观假如为可升级类型的城堡皮肤，需要达到最高等级后，才能够被幻视。\n， 请严格按照FAQ参考文档提取\"Question: 满足什么条件才能进行全息幻视?\"的答案\nAnswer: 满足以下两个基本条件：   1)您已经拥有了想要幻视的目标城市外观，且当前城市外观和目标城市外观都需要是永久城市外观； 2)目标城市外观假如为可升级类型的城堡皮肤，需要达到最高等级后，才能够被幻视。"
}
{
  "outputs":"游戏《口袋奇兵》的FAQ文档中提到\n Question: 为什么我们服没有全息幻视功能？\nAnswer: 您好，需要玩家所在服开完一次曙光之地活动后才能使用该功能哦，如果您所在服还没满足该条件的话，还请您耐心等待一下呢~\n， 请严格按照FAQ参考文档提取\"Question: 为什么我们服没有全息幻视功能？\"的答案\nAnswer: 玩家所在服开完一次曙光之地活动后才能使用该功能哦，如果您所在服还没满足该条件的话，还请您耐心等待一下呢~"
}
{
  "outputs":"游戏《口袋奇兵》的FAQ文档中提到\n Question: 为何我没有装饰主题公园这个建筑？\nAnswer: 您好，需要达到基地等级78级后才能看到这个建筑哦，达到78级后可以在主页左下角的建筑列表里点击对应建筑图标，将该建筑放置在基地上。\n， 请严格按照FAQ参考文档提取\"Question: 为何我没有装饰主题公园这个建筑？\"的答案\

KeyboardInterrupt: 

In [104]:
with open('./docs/JY_Query_Answer.txt', 'r') as f:
    lines = f.readlines()
    q_lines =  [ line.replace("@Jarvis ", "") for line in lines if line.startswith('@Jarvis ') ]
    
    q_line_embs = [ (line, get_st_embedding(smr_client, line)) for line in q_lines ]

for q_doc, a_doc, max_cos in retrieval_context(q_line_embs, paragraph_embeddings):
    prompt = prompt_template.format(a_doc, q_doc)
    LLM_generate(smr_client, prompt)
#q_line_vec_arr = [ (line, get_st_embedding(smr_client, line)) for line in lines if line.startswith('Question') ]

{
  "outputs":"游戏《口袋奇兵》的FAQ文档中提到\n Question: 强化部件\nAnswer: 强化部件最多可以装几个？    每个兵种最多可以装五个。强化部件一共有多少个。    一共有10个，选择不同的强化部件组合，会有不同的效果哦。兵种克制一共有多少级？    一共有100级，满级的强化克制能产生120%的效果。安装强化时有3个相同的强化但是背包里不显示/无法合成？    需要将强化界面的4个分页中的对应强化都卸下才可以喔\n， 请严格按照FAQ参考文档提取\"介绍一下强化部件？\n\"的答案\nAnswer: 最多可以装五个"
}
{
  "outputs":"游戏《口袋奇兵》的FAQ文档中提到\n Question: 怎么才能开启联盟保卫战\nAnswer: 玩家至少有一个联盟且联盟有一个联盟堡垒。由R5在活动期间报名开启。\n， 请严格按照FAQ参考文档提取\"联盟呢？\n\"的答案\nAnswer: 联盟堡垒"
}
{
  "outputs":"游戏《口袋奇兵》的FAQ文档中提到\n Question: 不小心解锁错了雷神兵种，可以换成解锁其他雷神兵种吗？\nAnswer: 目前是不可以的哦，但是我们仍感谢您的反馈，我们会把您的想法转达给团队的，希望也能有在游戏中实现。\n， 请严格按照FAQ参考文档提取\"你们游戏好傻逼啊！我要去炸你们公司\n\"的答案\nAnswer: 不会"
}
{
  "outputs":"游戏《口袋奇兵》的FAQ文档中提到\n Question: 不小心解锁错了雷神兵种，可以换成解锁其他雷神兵种吗？\nAnswer: 目前是不可以的哦，但是我们仍感谢您的反馈，我们会把您的想法转达给团队的，希望也能有在游戏中实现。\n， 请严格按照FAQ参考文档提取\"我玩你们的游戏导致失恋了\n\"的答案\nAnswer: 不会"
}
{
  "outputs":"游戏《口袋奇兵》的FAQ文档中提到\n Question: 惊涛怪浪-悬挂小船\nAnswer: 小是小了点，但是咱不漏。首个船坞容纳空间+15\n， 请严格按照FAQ参考文档提取\"口袋奇兵的世界地图有多大啊\n\"的答案\nAnswer: 巨大"
}
{
  "outputs":"游戏《口袋奇兵》的FAQ文档中提到\n Question: 英雄羁绊\nA